# Run a basic Hermes Hartree-Fock energy calculation on ligand conformers from Auto3d

In this notebook, we'll perform a hermes energy calculation on a ligand derived from a smiles string

# 0) Complete example
See the [sample notebook](/Quickstarts/auto3d_conformer_tautomer_generation-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path
import py3Dmol

import rush

## 1.1) Configuration

In [ ]:
EXPERIMENT = "tengu-py-auto3d_hermes"
LIGAND_SMILES = "CC(=O)OC1=CC=CC=C1C(=O)O "
LIGAND = "ASPRIN"
TAGS = ["qdx", EXPERIMENT, LIGAND]

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
import sys

os.chdir(WORK_DIR)

## 1.2) Build your client

In [ ]:
# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(batch_tags=TAGS)

In [ ]:
# |hide
client = await rush.build_provider_with_functions(batch_tags=TAGS, restore_by_default=True)

In [ ]:
# |hide
client.workspace = WORK_DIR
client.config_dir = WORK_DIR / ".rush"

# 2) Preparation
We want to convert our raw files into prepared qdxfs, with correct charges and missing residues filled

In [ ]:
help(client.auto3d)

Help on function auto3d in module rush.provider:

async auto3d(*args: *tuple[RushObject[NoneType], str, Record], target: 'Target | None' = None, resources: 'Resources | None' = {'storage': 18, 'storage_units': 'MB', 'gpus': 1}, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[NoneType], RushObject[list[Record]]]
    Generate 3D conformers from SMILES strings and other inputs
    
    Module version:  
    `github:talo/tengu-auto3d/5f1d10f3a8eca6d5c3d1313a4c5b710a91d6db9b#auto3d_tengu`
    
    QDX Type Description:
    
        molecule_file: Object{path: @$Bytes,  size: u32};
        molecule_file_type: string;
        options: Auto3dOptions{
            capacity: u32?,
             max_confs: u32?,
             memory: u32?,
             enumerate_isomer: bool?,
             enumerate_tautomer: bool?,
             k: i32?,
             convergence_threshold: f32?,
             opt_steps: u32?,
             threshold: f32?,
             patience: u32

In [ ]:
ligand_path = client.workspace / "aspirin.smiles"
with open(ligand_path, "w") as ligand_file:
    print(f"{LIGAND_SMILES} {LIGAND_SMILES}", file=ligand_file)

(conformer_sdf, conformer_qdxf) = await client.auto3d(
    ligand_path, "smi", {"k": 5, "use_gpu": True}, resources={"gpus": 1, "storage": "5", "storage_units": "GB"},
    target="NIX_SSH_2"
)

print(f"{datetime.now().time()} | Running ligand preparation!")

2024-02-25 16:50:01,515 - rush - INFO - Trying to restore job with tags: ['qdx', 'tengu-py-auto3d_hermes', 'ASPRIN'] and path: github:talo/tengu-auto3d/5f1d10f3a8eca6d5c3d1313a4c5b710a91d6db9b#auto3d_tengu
2024-02-25 16:50:01,567 - rush - INFO - Restoring job from previous run with id 28e91c65-d22b-4b34-9ee0-fc1df4774b42
16:50:01.567659 | Running ligand preparation!


In [ ]:
conformer_qdxf.source

UUID('28e91c65-d22b-4b34-9ee0-fc1df4774b42')

In [ ]:
try:
    await conformer_sdf.download(filename="01_prepared_ligand.sdf")
except FileExistsError:
    # we will raise an error if you try to overwrite an existing file; you can
    # force the file to overwrite by passing an absolute filepath instead
    pass

You should visualize your prepared ligand to spot check any issues

In [ ]:
view = py3Dmol.view()
with open(client.workspace / "objects" / "01_prepared_ligand.sdf", "r") as f:
    view.addModel(f.read(), "sdf")
    view.setStyle({"stick": {}})
    view.zoomTo()
    view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# 3) Quantum energy calculation
Finally, we submit our fragmented protein for quantum energy calculation, with custom configuration.

In [ ]:
help(client.hermes_energy)

Help on function hermes_energy in module rush.provider:

async hermes_energy(*args: *tuple[RushObject[Record], Record, typing.Optional[Record], typing.Optional[Record]], target: 'Target | None' = None, resources: 'Resources | None' = {'storage': 1034, 'storage_units': 'MB', 'gpus': 4}, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[Record]]
    Runs a HERMES energy calculation given a topology, and optionally model and keyword configurations.
    Will use the default model and keywords if none are provided
    
    Module version:  
    `github:talo/tengu-prelude/4a40d0bcbc623273ffc2a6af87f132e606fb6c92#hermes_energy`
    
    QDX Type Description:
    
        input: Object{
            path: @Conformer{
                subunits: [[u32]]?,
                 amino_acid_labels: [Label{u32,  string}]?,
                 residue_labels: [Label{u32,  string}]?,
                 residue_insertion_codes: [string]?,
                 topology: Topology{
     

In [ ]:
HERMES_RESOURCES = {
        "gpus": 4,
        "storage": 100,
        "storage_units": "MB",
        "walltime": 60 * 4,
}

In [ ]:
(conformer,) = await client.pick_conformer(conformer_qdxf, 0)

2024-02-25 16:50:05,622 - rush - INFO - Trying to restore job with tags: ['qdx', 'tengu-py-auto3d_hermes', 'ASPRIN'] and path: github:talo/tengu-prelude/c35cc6bbdfd547579d26aa898e91094424254459#pick_conformer
2024-02-25 16:50:05,674 - rush - WARNING - Multiple module instances found with the same tags and path
2024-02-25 16:50:05,674 - rush - WARNING - Multiple module instances found with the same tags and path
2024-02-25 16:50:05,716 - rush - INFO - Restoring job from previous run with id a9026857-e34e-43ed-ad9e-b066662576d7


In [ ]:
conformer_out = json.load(open(await conformer.download(), "r"))

In [ ]:
(hermes_energy,) = await client.hermes_energy(
    conformer,
    {},
    {
        "method": "RestrictedRIMP2",
        "basis": "cc-pVDZ",
        "aux_basis": "cc-pVDZ-RIFIT",
        "force_cartesian_basis_sets": True,
        "standard_orientation": "FullSystem",
    },
    {
    "scf": {
        "max_iter": 50,
        "max_diis_history_length": 12,
        "convergence_metric": "DIIS",
        "convergence_threshold": 0.000001,
        "density_threshold": 1e-10,
        "density_basis_set_projection_fallback_enabled": True,
        },
    "frag": {
        "cutoffs": {"dimer": 22},
        "cutoff_type": "Centroid",
        "level": "Dimer",
        "reference_fragment": len(conformer_out["topology"]["fragments"])
        - 1,
        },
    },
    #target="GADI",
    resources=HERMES_RESOURCES,
)

2024-02-25 16:50:09,614 - rush - INFO - Trying to restore job with tags: ['qdx', 'tengu-py-auto3d_hermes', 'ASPRIN'] and path: github:talo/tengu-prelude/4a40d0bcbc623273ffc2a6af87f132e606fb6c92#hermes_energy


In [ ]:
await hermes_energy.get()

2024-02-25 16:50:09,834 - rush - INFO - Argument ed318ef7-51fc-46f8-a270-7d83eb5e8dca is now ModuleInstanceStatus.RESOLVING
2024-02-25 16:50:10,921 - rush - INFO - Argument ed318ef7-51fc-46f8-a270-7d83eb5e8dca is now ModuleInstanceStatus.ADMITTED
2024-02-25 16:50:15,268 - rush - INFO - Argument ed318ef7-51fc-46f8-a270-7d83eb5e8dca is now ModuleInstanceStatus.DISPATCHED
2024-02-25 16:50:16,356 - rush - INFO - Argument ed318ef7-51fc-46f8-a270-7d83eb5e8dca is now ModuleInstanceStatus.RUNNING
2024-02-25 16:50:27,866 - rush - INFO - Argument ed318ef7-51fc-46f8-a270-7d83eb5e8dca is now ModuleInstanceStatus.AWAITING_UPLOAD


{'url': 'https://storage.googleapis.com/qdx-store/062b0efc-aae6-4d01-965a-6524e79293fb?x-goog-signature=1fbbaae1a435b9f53034118dcaf4b6b725477ee4eb97451b33aa99b9b89114d20498e2b9d8d3e5dd650e70b885dd8651245b9ebe28eda7d4425b327666e6606cd7d6522e4dc73fa5abe8a277ff7ba225bdc0d85be058538eaec380292e302a820fc5be4688adf848c0ddefca3cc0d0d7170e34878b7a64d49a0097de6266f3c1b13037520504f9f03c01682de498ac8ad3e0974d976c2b1a326c50ac16c866218c2ce00ea15c16f05320917eb704bbf1ed2de2123f50e0f98fe4370ee5fcbc65e3929f54684d6bb8d19c28133e576244d75b06ddc97a433056b291d60367d692b12e44142dfda96a48aa608d6b78e50a6e72ef49683032c73c526ca2c1207ce1&x-goog-algorithm=GOOG4-RSA-SHA256&x-goog-credential=qdx-store-user%40humming-bird-321603.iam.gserviceaccount.com%2F20240225%2Fasia-southeast1%2Fstorage%2Fgoog4_request&x-goog-date=20240225T085041Z&x-goog-expires=3600&x-goog-signedheaders=host'}